# Diamond prices

We've been given a data set containing the prices and attributes of almost 54,000 diamonds. Let's take a dig into this data and see what we can pull out...

## Problem

We want to **predict** the **price** of diamond given its **attributes**.

## Step 1: Data exploration

Let's start with the data exploration step, and figure out what is the most important variable to predict the price of a diamond.

In [ ]:
# library to interact with your system
import os
# Module for manipulating paths
from os import path

# Library to access and manipulate the data
import pandas as pd
# Libraries to visualize the data
import seaborn as sns
import matplotlib.pyplot as plt

### Import

Before we can really do anyhing useful we need to load up the data. We've put all the data for this into the data directory - let's take a look at the files we've got 

In [ ]:
# Let's find out where we're execting this at the moment
os.getcwd()

Our data are in the data folder one directory above the notebook directory.

In [ ]:
# '..' means the directory above this one
data_folder = path.join(path.abspath('..'), 'data')
data_folder

In [ ]:
os.listdir(data_folder)

Pandas will do most of the loading for us from a range of formats, from csv files to Excel spreedsheats. Let's load up the diamonds data.

In [ ]:
# This points to the location of the fault data file on my computer
diamond_file = path.join(data_folder, 'diamonds.csv')

# The first column in the file is the index column, let pandas know
df = pd.read_csv(diamond_file, index_col=0)

Let's take a look at some basic information about this dataframe.

In [ ]:
df.info()

So, we have 53940 samples, and 10 variables.

What that doesn't tell us is the meaning of each variable:

- **carat**: Weight of the diamond
- **cut**: Quality of the cut of the diamond
- **color**: Color of the diamond
- **clarity**: Absence of inclusions or blemish in the diamond
- **depth**: Height of the diamond, expressed as percent of the diamond's average diameter
- **table**: Width of the diamond's table (the flat top area), expressed as percent of the diamond's average diameter
- **price**: Price of the diamond
- **x**: Length of the diamond in mm
- **y**: Width of the diamond in mm
- **z**: Height of the diamond in mm

### Tidy

Let's check if the dataframe is tidy.

In [ ]:
df.head()

Looks like it is: each column correspond to a variable, none of the variable names carry extra data.

Let's have a look at any missing data

In [ ]:
df.isnull().sum()

No explicit missing values in the data set.

### Explore

Let's start with an easy question.

#### Question 1: What are the ranges for each variables?

In [ ]:
df.describe()

The price seems to vary quite a bit, so does the carat. x, y, and z look weird though...

#### Question 2: How can x, y, or z be equal to 0?

Let's plot their distribution to have a better look.

In [ ]:
df.hist(column=['x', 'y', 'z'], bins=100, figsize=(12, 12))

x and y are globally larger than z, which is consistent with the definition and the common shape of diamond, usually wider than deep. However, there is no consistency between the x, y, or z equal to 0 and the other attributes.

In [ ]:
df[(df['x'] == 0) | (df['y'] == 0) | (df['z'] == 0)]

Values near 0 look like outliers, they might actually be missing values. And y and z seem to have some weird values on the other side of their ditributions...

#### Question 3: How can y and z have so high values?

On the highest side of the range, x's distribution seems fine, and indeed the values from the other attributes look normal.

In [ ]:
df.sort_values(by=['x'], ascending=False).head()

y's and z's distributions seem to show outliers, and indeed the values from the other attributes look inconsistent (compare x and y in particular).

In [ ]:
df.sort_values(by=['y'], ascending=False).head()

In [ ]:
df.sort_values(by=['z'], ascending=False).head()

Since there should be a relationship between x, y, and z, we can also check for outliers using scatter plots.

In [ ]:
df.plot(x='x', y='y', kind='scatter')

In [ ]:
df.plot(x='x', y='z', kind='scatter')

In [ ]:
df.plot(x='y', y='z', kind='scatter')

Now we clearly see that a few samples behave differently than the others. Let's remove them.

#### Question 4: What happens when we remove outliers?

In [ ]:
df_clean = df[(0 < df['x']) & (0 < df['y']) & (df['y'] < 11) & (0 < df['z']) & (df['z'] < 11)]

In [ ]:
df_clean.describe()

In [ ]:
df_clean.hist(column=['x', 'y', 'z'], bins=100, figsize=(12, 12))

Now we can see much more consistency between x, y, and z.

It seems like we can move on to the other variables. Let's see how they all vary.

#### Question 5: What are the variations within the variables?

In [ ]:
df_clean.hist(bins=200, figsize=(12, 12))

As the price increases, we have fewer and bigger diamonds, which seem to make sense. The carat in particular seems to follow a similar distribution. The carat is often associated with wealth, so it would make sense to have a close relationship. Let's have a closer look.

#### Question 6: Is there a correlation between price and carat?

Lots of points are close, so a scatter plot can be a big mess. Let's add some transparency (parameter *alpha*).

In [ ]:
df.plot(x='carat', y='price', kind='scatter', figsize=(12, 12), alpha=0.05)

We see clear jumps from the carat distribution, and each jump leads to an increase in price, at least globally. Looking at 1, 1.5, or 2, it seems like rounded values are more attractive.

So, there is a clear correlation between price and carat, but more information would be required to accurately predict a price, since two diamonds with the same carat could have very different prices.

#### Question 7: Which other variables might help?

For this question we're switching to more complex plots, so to seaborn instead of pandas for visualization.

In [ ]:
g = sns.PairGrid(df_clean)
g.map_diag(plt.hist, bins=100)
g.map_offdiag(plt.scatter, s=5, alpha=0.05)

Here we see the relationships between all the variables, but we're more particularly interested in the row with the price as y axis.

Depth and table don't seem particularly informative. x, y, and z follow a relationship pretty similar to that of carat. Another way to to look at those relationships is through a correlation matrix.

In [ ]:
corr = df_clean.corr()
sns.heatmap(data=corr, square=True , annot=True, cbar=True)

The correlation between price and carat is the highest, confirming our initial guess. Even if their correlation is pretty good, x, y, and z don't bring any more information when looking at their scatter plot. Depth seems useless to predict a price; table doesn't seem much more useful.

But we have ignored three variables, because they are discrete and not continuous. Let's look at them.

#### Question 8: What is the relationship between the price and the discrete variables?

Let's use boxplots to compare the price to the cut, color, and clarity.

In [ ]:
df_clean.boxplot(by='cut', column=['price'])

In [ ]:
df_clean.boxplot(by='color', column=['price'])

In [ ]:
df_clean.boxplot(by='clarity', column=['price'])

While there are correlations, they don't seem as strong as the carat. The color seems to have the higher impact, let's have a closer look.

In [ ]:
sns.relplot(x='carat', y='price', data=df_clean, hue='color', alpha=0.25, edgecolor=None)

In [ ]:
sns.relplot(x='carat', y='price', data=df_clean, col='color', col_wrap=4, alpha=0.25, edgecolor=None)

Looking at all the data, it doesn't feel like there is a clear relationship. Let's look at everything at the same time.

In [ ]:
sns.relplot(x='carat',
            y='price',
            data=df_clean,
            col='color',
            row='clarity',
            hue='cut',
            alpha=0.25,
            edgecolor=None)

Looks like color and clarity combined give simpler relationships between carat and price. Some color and clarity values lead to high prices even at low carat.

Now we have a better idea of what drives the price of a diamond: The carat is the most important attribute, and the color and clarity should improve the predictions.

Before moving on to the next step, let's save our cleaned dataset.

#### Save the cleaned dataset

In [ ]:
diamond_file_clean = path.join(data_folder, 'diamonds_clean.csv')
df_clean.to_csv(diamond_file_clean)

And we can read it straight back in.

In [ ]:
pd.read_csv(diamond_file_clean).info()

## Step 2: Modeling

Now let's have a look at how well we can actually predict a diamond's price using its carat.

In [ ]:
# Module to split the data
from sklearn.model_selection import train_test_split
# Module for linear regression
from sklearn.linear_model import LinearRegression
# Modules to transform the variables for regression
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import mean_squared_error

### Data preparation

Let's define the input (x) and output (y) of our model.

In [ ]:
x = df_clean['carat'].values.reshape(-1, 1)
y = df_clean['price'].values

We keep 25% of our data as test set. The rest corresponds to our training data.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=100)

### Model 2

We start with the simplest model possible: a straight line.

In [ ]:
model = LinearRegression()

Let's train the model on the training set.

In [ ]:
model.fit(x_train, y_train)

Now let's look at the score of our model on the test set (0 being the worst, 1 the best score).

In [ ]:
model.score(x_test, y_test)

We can plot the model on top of the data to get a better idea at what the model can do.

In [ ]:
x_plot = np.linspace(0.2, 3, 100).reshape(-1, 1)
y_plot = model.predict(x_plot)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 12))
df.plot(x='carat', y='price', kind='scatter', alpha=0.05, ax=ax)
ax.plot(x_plot, y_plot, color='red')

Or even better, we can compute the residuals on the test set.

In [ ]:
y_pred = model.predict(x_test)
y_res = y_pred - y_test

In [ ]:
sns.distplot(y_res, bins=100, kde=False, axlabel='Residuals')

In [ ]:
mean_squared_error(y_test, y_pred)

We can see that despite a decent score, the model shows large residuals.

### Model 1

The trend between carat and price is not linear, but slightly skewed. We can take that into account by training the regression on transformed data.

In [ ]:
model = TransformedTargetRegressor(regressor=LinearRegression(),
                                   transformer=QuantileTransformer())

Let's train the new model.

In [ ]:
model.fit(x_train, y_train)

Unfortunately, the score gets worse.

In [ ]:
model.score(x_test, y_test)

Let's have a closer look at the model.

In [ ]:
x_plot = np.linspace(0.2, 3, 100).reshape(-1, 1)
y_plot = model.predict(x_plot)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 12))
df.plot(x='carat', y='price', kind='scatter', alpha=0.05, ax=ax)
ax.plot(x_plot, y_plot, color='red')

And at the residuals, which seem also worse than with the linear model alone.

In [ ]:
y_pred = model.predict(x_test)
y_res = y_pred - y_test

In [ ]:
sns.distplot(y_res, bins=100, kde=False, axlabel='Residuals')

In [ ]:
mean_squared_error(y_test, y_pred)

At this stage, it seems that a model complex model won't do. What we need instead is more information, so more inputs. Trying to include the color and clarity would be an interesting next step.